# Car Price Prediction

### Description du problème
Le problème dans ce cas est lié aux marché des voitures, qui pourait etre utile pour une entreprise de fabrication de voiture qui souhaite s'inflitrer au sein du marché d'automobile au Etas-unis (US).
Le <a href="https://www.kaggle.com/goyalshalini93/car-data">Dataset utilisé</a> est une collection de données relatives aux voitures vendues au Etas unis (trouvé sur Kaggle).
Cette étude vise à comprendre les principaux facteurs influancant les prix des voitures aux USA. Les questions qu'on doit y répondre en conclusion de cette étude sont:
<ol>
        <li> Quels sont les variales qui influencent la prediction du prix dla voiture ?</li>
        <li> Quelle est la qualité de description du prix d'une voiture par ces variables ?</li>
</ol>

In [16]:
import findspark
findspark.init()

## Chargement des données

In [17]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TP1 BDM").getOrCreate()
df = spark.read.csv("./CarPrice_Assignment.csv",header=True)
df.take(1)

[Row(car_ID='1', symboling='3', CarName='alfa-romero giulia', fueltype='gas', aspiration='std', doornumber='two', carbody='convertible', drivewheel='rwd', enginelocation='front', wheelbase='88.6', carlength='168.8', carwidth='64.1', carheight='48.8', curbweight='2548', enginetype='dohc', cylindernumber='four', enginesize='130', fuelsystem='mpfi', boreratio='3.47', stroke='2.68', compressionratio='9', horsepower='111', peakrpm='5000', citympg='21', highwaympg='27', price='13495')]

## Exploration des données
### Types des champs

In [18]:
df.cache()
df.printSchema()

root
 |-- car_ID: string (nullable = true)
 |-- symboling: string (nullable = true)
 |-- CarName: string (nullable = true)
 |-- fueltype: string (nullable = true)
 |-- aspiration: string (nullable = true)
 |-- doornumber: string (nullable = true)
 |-- carbody: string (nullable = true)
 |-- drivewheel: string (nullable = true)
 |-- enginelocation: string (nullable = true)
 |-- wheelbase: string (nullable = true)
 |-- carlength: string (nullable = true)
 |-- carwidth: string (nullable = true)
 |-- carheight: string (nullable = true)
 |-- curbweight: string (nullable = true)
 |-- enginetype: string (nullable = true)
 |-- cylindernumber: string (nullable = true)
 |-- enginesize: string (nullable = true)
 |-- fuelsystem: string (nullable = true)
 |-- boreratio: string (nullable = true)
 |-- stroke: string (nullable = true)
 |-- compressionratio: string (nullable = true)
 |-- horsepower: string (nullable = true)
 |-- peakrpm: string (nullable = true)
 |-- citympg: string (nullable = true)
 |

On remarque que les données de ce jeu de données sont dr type chaine de caracteres, donc on doit procéder par une conversion de type (cast) pour rendre utilisable ces données là

### Analyse descriptive (1)

In [19]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
car_ID,205,103.0,59.32256456582661,1,99
symboling,205,0.8341463414634146,1.2453068281055295,-1,3
CarName,205,None,None,Nissan versa,vw rabbit
fueltype,205,None,None,diesel,gas
aspiration,205,None,None,std,turbo
doornumber,205,None,None,four,two
carbody,205,None,None,convertible,wagon
drivewheel,205,None,None,4wd,rwd
enginelocation,205,None,None,front,rear


#### On procède à une étape de réduction de colonnes en éliminant les colonnes qui ne sont pas trés utiles pour notre étude
### Analyse descriptive (2)

In [20]:
columns_to_drop=['CarName','fueltype','aspiration','doornumber','carbody','drivewheel','enginelocation','enginetype','cylindernumber','fuelsystem']
df=df.drop(*columns_to_drop)
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
car_ID,205,103.0,59.32256456582661,1,99
symboling,205,0.8341463414634146,1.2453068281055295,-1,3
wheelbase,205,98.75658536585378,6.0217756850255695,100.4,99.8
carlength,205,174.04926829268305,12.337288526555188,141.1,208.1
carwidth,205,65.90780487804875,2.145203852687182,60.3,72.3
carheight,205,53.724878048780525,2.443521969904905,47.8,59.8
curbweight,205,2555.5658536585365,520.6802035016384,1488,4066
enginesize,205,126.90731707317073,41.642693438179855,103,98
boreratio,205,3.3297560975609772,0.27084370542622915,2.54,3.94


Comme on peut le remarquer les données de ce dataset sont des données de type chaine de caracteres, donc une conversion de type est nécessaire

In [21]:
df=df.withColumn('car_ID',df['car_ID'].cast("float").alias('car_ID'))
df=df.withColumn('symboling',df['symboling'].cast("float").alias('symboling'))
df=df.withColumn('wheelbase',df['wheelbase'].cast("float").alias('wheelbase'))
df=df.withColumn('carlength',df['carlength'].cast("float").alias('carlength'))
df=df.withColumn('carwidth',df['carwidth'].cast("float").alias('carwidth'))
df=df.withColumn('carheight',df['carheight'].cast("float").alias('carheight'))
df=df.withColumn('curbweight',df['curbweight'].cast("float").alias('curbweight'))
df=df.withColumn('enginesize',df['enginesize'].cast("float").alias('enginesize'))
df=df.withColumn('boreratio',df['boreratio'].cast("float").alias('boreratio'))
df=df.withColumn('stroke',df['stroke'].cast("float").alias('stroke'))
df=df.withColumn('compressionratio',df['compressionratio'].cast("float").alias('compressionratio'))
df=df.withColumn('horsepower',df['horsepower'].cast("float").alias('horsepower'))
df=df.withColumn('peakrpm',df['peakrpm'].cast("float").alias('peakrpm'))
df=df.withColumn('citympg',df['citympg'].cast("float").alias('citympg'))
df=df.withColumn('highwaympg',df['highwaympg'].cast("float").alias('highwaympg'))
df=df.withColumn('price',df['price'].cast("float").alias('price'))
df.printSchema()

root
 |-- car_ID: float (nullable = true)
 |-- symboling: float (nullable = true)
 |-- wheelbase: float (nullable = true)
 |-- carlength: float (nullable = true)
 |-- carwidth: float (nullable = true)
 |-- carheight: float (nullable = true)
 |-- curbweight: float (nullable = true)
 |-- enginesize: float (nullable = true)
 |-- boreratio: float (nullable = true)
 |-- stroke: float (nullable = true)
 |-- compressionratio: float (nullable = true)
 |-- horsepower: float (nullable = true)
 |-- peakrpm: float (nullable = true)
 |-- citympg: float (nullable = true)
 |-- highwaympg: float (nullable = true)
 |-- price: float (nullable = true)



Calculons maintenant la corrélation entre les différentes variables de notre dataset, le Coefficient de Corrélation mesure la dépendance entre deux variables, en effet, ce coefficent prend ses valeurs entre -1 et 1, si ce coefficient est prés de 1 cela veut dire que les deux variables en question sont corrélées entre elles et donc dépendantes

In [22]:
import six
for i in df.columns:
    if not( isinstance(df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation entre price et ", i, df.stat.corr('price',i))

Correlation entre price et  car_ID -0.10909334037698282
Correlation entre price et  symboling -0.07997822501359858
Correlation entre price et  wheelbase 0.577815609013954
Correlation entre price et  carlength 0.6829200061793368
Correlation entre price et  carwidth 0.7593252732789826
Correlation entre price et  carheight 0.11933627096290873
Correlation entre price et  curbweight 0.8353048796203731
Correlation entre price et  enginesize 0.8741448022848783
Correlation entre price et  boreratio 0.5531732639743967
Correlation entre price et  stroke 0.07944309329818429
Correlation entre price et  compressionratio 0.06798351616221464
Correlation entre price et  horsepower 0.8081388231007026
Correlation entre price et  peakrpm -0.0852671497816066
Correlation entre price et  citympg -0.6857513366309157
Correlation entre price et  highwaympg -0.6975990921640883
Correlation entre price et  price 1.0


Procédons maintenant à la vectorisation de nos données, une étape dans laquelle on va considere tous les champs de chaque ligne du dataset comme étant un vecteur, mis à part le champs prix (price) qui lui consiste en notre champs d'étude (Target variable)

In [23]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler=VectorAssembler(inputCols=['car_ID','symboling','wheelbase','carlength','carwidth','carheight','curbweight','enginesize','boreratio','stroke','compressionratio','horsepower','peakrpm','citympg','highwaympg','price'],outputCol='features')
vect_df=vectorAssembler.transform(df)
vect_df=vect_df.select(['features','price'])
vect_df.toPandas()

,features,price
0,"[1.0, 3.0, 88.5999984741211, 168.8000030517578...",13495.0
1,"[2.0, 3.0, 88.5999984741211, 168.8000030517578...",16500.0
2,"[3.0, 1.0, 94.5, 171.1999969482422, 65.5, 52.4...",16500.0
3,"[4.0, 2.0, 99.80000305175781, 176.600006103515...",13950.0
4,"[5.0, 2.0, 99.4000015258789, 176.6000061035156...",17450.0
...,...,...
200,"[201.0, -1.0, 109.0999984741211, 188.800003051...",16845.0
201,"[202.0, -1.0, 109.0999984741211, 188.800003051...",19045.0
202,"[203.0, -1.0, 109.0999984741211, 188.800003051...",21485.0
203,"[204.0, -1.0, 109.0999984741211, 188.800003051...",22470.0


### Régression lineaire

Pour procéder à l'effectuation de la régression lineaire, on doit d'abord découper notre data set en deux partie (70% pour la partie entrainement, et le 30% restant pour la partie test).
Notre modele sera ainsi conçu en se basant sur les 70% de données (données d'entrainement)
Y=A1x1+A2x2+...+Anxn+B

In [24]:
splits=vect_df.randomSplit([0.7,0.3])
train_df=splits[0]
test_df=splits[1]
from pyspark.ml.regression import LinearRegression
linear=LinearRegression(featuresCol='features',labelCol='price',maxIter=10, regParam=0.3,elasticNetParam=0.8)
linear_model=linear.fit(train_df)
print("Les coefficients (parametres An) sont : "+str(linear_model.coefficients))
print('Le B est égale à : '+str(linear_model.intercept))

Les coefficients (parametres An) sont : [-4.014440330633357,-55.18186364133474,0.0,-12.325284339197394,-64.39561114547621,121.71728835059311,0.5018134419936087,31.45185746260674,-57.59664648586266,-1317.042982261776,89.84188992489936,14.213118516946974,0.9116210627146937,-6.373541937323077,24.33750907906415,0.7720218198562527]
Le B est égale à : -5125.01535803601


In [25]:
synthese=linear_model.summary
print('RMSE : '+str(synthese.rootMeanSquaredError))
print('R² : '+str(synthese.r2))

RMSE : 793.0118143583425
R² : 0.989891439744169


In [26]:
train_df.describe().toPandas()

,summary,price
0,count,149
1,mean,13068.46085885067
2,stddev,7914.0225231771365
3,min,5118.0
4,max,41315.0


RMSE mesure les différences entre les valeurs prédites par le modèle et les valeurs réelles. Cependant, RMSE seul n'a pas de sens tant que nous ne comparons pas avec la valeur «price» réelle, telle que la moyenne, min et max. Après une telle comparaison, notre RMSE semble plutôt bon.

R au carré à 0,989 indique que dans notre modèle, environ 99% de la variabilité de «price» peut être expliquée à l'aide du modèle. C'est optimale. Cependant, nous devons faire attention au cas ou les performances sur l'ensemble d'entraînement ne constituent pas une bonne approximation des performances sur l'ensemble de test.

In [38]:
linear_predictions=linear_model.transform(test_df)
linear_predictions.select("prediction",'price','features').show(5)
from pyspark.ml.evaluation import RegressionEvaluator
linear_evaluator=RegressionEvaluator(predictionCol="prediction",labelCol='price',metricName='r2')
print("R² sur les données de test = %g"%linear_evaluator.evaluate(linear_predictions))

+------------------+-------+--------------------+
|        prediction|  price|            features|
+------------------+-------+--------------------+
|13960.829355810838|13495.0|[1.0,3.0,88.59999...|
|15224.527404215598|15250.0|[6.0,2.0,99.80000...|
|18127.549480761463|18920.0|[8.0,1.0,105.8000...|
|23049.263366573272|24565.0|[15.0,1.0,103.5,1...|
| 6685.810214497994| 6377.0|[23.0,1.0,93.6999...|
+------------------+-------+--------------------+
only showing top 5 rows

R² sur les données de test = 0.990818


In [35]:
resultat_test=linear_model.evaluate(test_df)
print("RMSE sur les données de test = %g"%resultat_test.rootMeanSquaredError)

RMSE sur les données de test = 781.663


In [ ]:
On a eu un R² optimale aussi sur les données de test

In [36]:
print("Le nombre d'itérations effectués = %d"%synthese.totalIterations)
print("Les residus : ")
synthese.residuals.show(10)

Le nombre d'itérations effectués = 10
Les residus : 
+-------------------+
|          residuals|
+-------------------+
| 223.25951585175426|
| -636.0884012320857|
|  383.5743406436268|
| 256.18059088298105|
|  567.7819595532674|
| 1689.8326759860975|
| 131.82654969593204|
|-14.150458796946623|
| -7.650546577828209|
|  624.2362566419324|
+-------------------+
only showing top 10 rows



In [37]:
predictions=linear_model.transform(test_df)
predictions.select("price",'features','prediction').show()

+-------+--------------------+------------------+
|  price|            features|        prediction|
+-------+--------------------+------------------+
|13495.0|[1.0,3.0,88.59999...|13960.829355810838|
|15250.0|[6.0,2.0,99.80000...|15224.527404215598|
|18920.0|[8.0,1.0,105.8000...|18127.549480761463|
|24565.0|[15.0,1.0,103.5,1...|23049.263366573272|
| 6377.0|[23.0,1.0,93.6999...| 6685.810214497994|
| 6229.0|[25.0,1.0,93.6999...|6584.8435771863815|
| 8558.0|[28.0,1.0,93.6999...|  8692.12273399318|
|12964.0|[30.0,3.0,95.9000...|12987.711576824022|
| 6479.0|[31.0,2.0,86.5999...|6087.2254213233855|
| 6855.0|[32.0,2.0,86.5999...| 7465.851620901592|
| 5399.0|[33.0,1.0,93.6999...|6037.3762549536505|
| 6529.0|[34.0,1.0,93.6999...| 7377.163525820218|
| 7295.0|[36.0,0.0,96.5,16...|8116.9164624240175|
|10345.0|[43.0,1.0,96.5,16...| 10068.84577148332|
|13645.0|[58.0,3.0,95.3000...|11855.112819756836|
|15645.0|[59.0,3.0,95.3000...|14256.988711107559|
|10795.0|[64.0,0.0,98.8000...|11617.446507004988|
